<font size=25>Laboratory 2 summary</font>

In this lab you will:
* Learn how the `torch.autograd` package tracks the computation graph and performs backpropagation
* Train a Linear Regressor using Gradient Descent
* Define a Neural Network by hand and train it on a dataset
* Learn how to use `torch.nn.Module` and all its adjacent helpers to define a Neural Network

# **Part I: Autograd**

|![Computational Graph](https://i.ibb.co/zr7TV4H/computational-graph.png)|
|:--:|
| Example of a simple computational graph. Autograd computes df/dx, df/dy, df/dz. [Source: CS231, lecture 4](http://cs231n.stanford.edu/slides/2019/cs231n_2019_lecture04.pdf) |

The **torch.autograd** package provides automatic differentiation for all operations on Tensors. It is a *define-by-run* framework (the computational graph is *dynamic*), which means that your backprop is defined by how your code is run, and that every single iteration can be different (as opposed to TensorFlow, where you first define the graph and then run it - a *static graph*).

If you set the Tensor attribute `.requires_grad` to `True`, it starts to track all operations on it. When you finish your computation you can call `.backward()` and have all the gradients computed automatically. The gradient for this tensor will be accumulated into `.grad` attribute.

There’s one more class which is very important for autograd implementation - a `Function`.

`Tensor` and `Function` are interconnected and build up an acyclic graph, that encodes a complete history of computation. Each tensor has a `.grad_fn` attribute that references a Function that has created the Tensor (except for Tensors created by the user - their `grad_fn` is `None`).

If you want to compute the derivatives, you can call `.backward()` on a Tensor. If Tensor is a scalar (i.e. it holds a one element data), you don’t need to specify any arguments to `backward`, however if it has more elements, you need to specify a gradient argument that is a tensor of matching shape.

*Ref: [autograd](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#sphx-glr-beginner-blitz-autograd-tutorial-py)*

We can use `torchviz` to visualise the computation history. It displays the computation graph, coloring the leaf nodes in blue, the root (the element whose graph we are plotting in green), and all other intermediate operations in grey.

First we would need to install and import it in this notebook:

In [ ]:
!pip install torchviz

In [ ]:
from functools import partial
from IPython.display import HTML
import math
import matplotlib.pyplot as plt
from matplotlib import animation, rc
import numpy as np
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchviz
from typing import Iterator

## Viewing the computational graph

Let us define a simple Tensor, specifying the `requires_grad` parameter, and then draw the computation graph:

In [ ]:
a = torch.ones(2, 2, requires_grad=True)
print("a = ", a)

In [ ]:
b = torch.zeros(2, 2, requires_grad=True)
c = a + b
print("b = ", b)
print("c = ", c)

Notice how the `grad_fn` attribute is now populated with `AddBackward0`. This means that `autograd` knows that the tensor `c` results from the addition operation between tensor `a` and `b`. To compute the gradients, autograd will traverse the computational graph from the root node (in this case, `c`) to its leaves (`a` and `b`).

Let's visualize the graph:

In [ ]:
torchviz.make_dot(c)

Note that, had we not specified `requires_grad`, the computation history would not have been tracked.

In [ ]:
a = torch.ones(2, 2)
b = torch.zeros(2, 2)
c = a + b
print("c = ", c)
torchviz.make_dot(c)

Let's look at another example, involving more tensors:

In [ ]:
a = torch.randn(5, 2, requires_grad=True)
b = torch.randn(2, 7, requires_grad=True)
c = torch.randn(5, 7, requires_grad=True)

x = torch.tanh(a @ b + c).sum()

print(x)
torchviz.make_dot(x)

## Computing the gradient of the result with respect to all tensors in the graph
Let's consider a simple dot product multiplication
 $y = \textbf{w}^T \textbf{x}$ and plot it's computational graph:

In [ ]:
w = torch.rand([3,1], requires_grad=True)
x = torch.Tensor([[10, 11, 12]])
y = x @ w # equivalent to y = torch.matmul(w, x)
print(f'w = {w}')
print(f'x = {x}')
print(f'y = {y}')
torchviz.make_dot(y)

We compute the gradient of y with respect to **w**: $\nabla_
\textbf{w} y = [\frac{\partial y}{\partial w_1}, \frac{\partial y}{\partial w_2}, \frac{\partial y}{\partial w_3}] = [x_1, x_2, x_3] = \textbf{x}$

To do this in PyTorch, we simply call `backward()` on Tensor `y`:


In [ ]:
y.backward()
print(y)

The gradient of y with respect to **w** will be stored in `w.grad`:



In [ ]:
print("Gradient of y wrt w = ", w.grad)
print("Gradient of y wrt x = ", x.grad)

What is the gradient of y with respect to **x** and why was it not stored in `x.grad`?

**Important:** Calling `.backward()` erases the forward graph by default. Calling `.backward()` again will raise an error.

In [ ]:
try:
    y.backward()
except RuntimeError as e:
    print(e)

There are times when this is undesirable:

In [ ]:
a = torch.rand([1,3], requires_grad=True)
b = torch.rand([3,2])
c = torch.rand([1])

node1 = (a @ b).sum()
node2 = node1 * c

print(f'node1.shape = {node1.shape}')

print("node1 graph:")
torchviz.make_dot(node1)

In [ ]:
print(f'node2.shape = {node2.shape}')
print("node2 graph:")
torchviz.make_dot(node2)

Notice how the first graph is a subgraph of the second one. When calling the first `.backward()`, the computational graph will get reset.

In [ ]:
node1.backward() # now the computation graph gets reset
node2.backward() # this will throw the error

To override this behavior, you can call `.backward(retain_graph=True)`.

In [ ]:
a = torch.rand([1,3], requires_grad=True)
b = torch.rand([3,2])
c = torch.rand([1])

node1 = (a @ b).sum()
node2 = node1 * c

print(f'node1.shape = {node1.shape}')
print(f'node2.shape = {node2.shape}')

node1.backward(retain_graph=True) # Prevent the erasure of the computation graph
print(a.grad)
node2.backward() # Now this will work
print(a.grad)

We might want to perform operations to our Tensors that do not contribute to the training of the model (the most common example is calculating the loss functions for test data while training). We can make sure that the computational graph is not stored by wrapping our code in `torch.no_grad()`:

In [ ]:
a = torch.randn(3, requires_grad=True)
with torch.no_grad():
  b = a + 1

torchviz.make_dot(b)

## Exercise 1: Linear regression with autograd

Recall the linear regression exercise - in our previous lab we've trained our model by computing the closed-form solution. Let's do a linear regression using Gradient Descent.

We'd like to find the global minimum of the error function $E(w)$ - i.e. we want to find a *weight vector $w$* which minimizes $E(w)$ (\*).

Let $\tau$ be our iteration step. We want to perform *Gradient Descent* on the loss surface - the simplest approach to using gradient information is to update the weights by making small steps in the direction of the negative gradient (see Chapter 5.2 - Bishop) [[1]]: 

$$w^{\tau+1}=w^\tau - \eta\nabla E(w^\tau)$$

The $\eta$ parameter is known as the *learning rate* and it represents the magnitude of the gradient step.

We're going to use the *Mean Squared Error (MSE)* loss for the Linear Regression. Recall that the MSE Loss is defined as follows:

$$MSE(y, \hat{y}) = \frac{1}{n} \sum_{i=0}^{n} (y_i - \hat{y_i})^2 $$

Where $n$ is the dataset length, $y$ is the true label vector and $\hat{y}$ is the prediction vector.

(\*): *In the context of Neural Networks, this won't be always possible (or desirable - recall the polynomial that was overfitting our data in the previous lab). The loss surface of a Neural Network is a lot more complex - there could be saddle points, parts of the surface with almost no gradients or highly irregular neighbours. We'll (usually) find a "good enough" local minima, improving our chances of finding it by using various optimization tehniques.*

[1]: http://users.isr.ist.utl.pt/~wurmd/Livros/school/Bishop%20-%20Pattern%20Recognition%20And%20Machine%20Learning%20-%20Springer%20%202006.pdf

**TODO:**

1) Implement the `__call__()` method of the `MSE` class. This method will compute the MSE Loss between predictions in `y` and ground-truth values in `target`

2) Implement the `forward()` method for the `GDLinearRegression` class. This method will predict our points w.r.t. the weights and biases.

3) Impement the `step()` method for the `GD` class. This method will update perform a gradient descend update to the parameters.

4) Implement the `train()` routine:

- Make a prediction with your `GDLinearRegression` model.
- Calculate the MSE Loss using the `MSE` object.
- Calculate the gradients by using `.backward()` on the loss.
- Do an optimizer `.step()` and reset the gradients.


In [ ]:
class MSE():
  """The Mean Squared Error loss"""
  
  def __call__(self, y: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """
    TODO: compute the MSE between predictions in tensor y and ground truth 
    values in tensor target. Both tensors are unidimensional.
    y: tensor of size N containing the predictions 
    target: tensor of size N containing the ground-truth values
    """
    mse = ... # Todo
    return mse

Note that we're inheriting `nn.Module`. In this specific case, the only reason for doing this is that the `nn.Module` class provides us with the `.parameters()` class method (used when instantiating the optimizer). 

Note: In the last exercise you will also inherit `nn.Module`.

In [ ]:
class GDLinearRegression(nn.Module):
  """A simple Linear Regression model"""

  def __init__(self):
    super().__init__()
    # We're initializing our model with random weights
    self.w = nn.Parameter(torch.randn(1, requires_grad = True))
    self.b = nn.Parameter(torch.randn(1, requires_grad = True))

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    """
    TODO: implement feedforwad call for a simple linear regression:
      y = wx + b
    Arguments: x is tensor of size (num_examples x 1)
    """
    y = ... # Todo
    return y

  # PyTorch is accumulating gradients
  # After each Gradient Descent step we should reset the gradients
  def zero_grad(self):
    self.w.grad.zero_()
    self.b.grad.zero_()

In [ ]:
class GD:
  """
  Gradient Descent optimizer
  We will write our own class for now, but in the future we will use
  the Optimizers implemented in Pytorch:
  https://pytorch.org/docs/stable/optim.html#torch.optim.Optimizer 
  """
  def __init__(self, params: Iterator[nn.Parameter], lr: float):
    self.w, self.b = list(params)
    self.lr = lr

    # We'll use these two for a plot :)
    if len(self.w.shape) == 1:
      self.w_hist = [self.w.item()]
      self.b_hist = [self.b.item()]

  def step(self):
    """
    Perform a gradient decent step. Update the parameters w and b by using:
     - the gradient of the loss with respect to the parameters
     - the learning rate
    This method is called after backward(), so the gradient of the loss wrt 
    the parameters is already computed (but where is it stored?)
    """
    with torch.no_grad():
      self.w -= ... # Todo
      self.b -= ... # Todo

    # We'll use these two for a plot :)
    if len(self.w.shape) == 1:
      self.w_hist.append(self.w.item())
      self.b_hist.append(self.b.item())

In [ ]:
def train(model: GDLinearRegression, data: torch.Tensor,
          labels: torch.Tensor, optim: GD, criterion: MSE):
  """Linear Regression train routine"""
  # TODO forward pass: compute predictions (hint: use model.forward)
  predictions = ... 
  
  # TODO forward pass: compute loss (hint: use criterion)
  loss = ... 

  # TODO backpropagation: compute gradients of loss wrt weights
  # ...
  
  # TODO GD step: update weights using the gradients (hint: use optim)
  # ...

  # TODO reset the gradients (hint: use model)
  # ...
  
  return model

Let's get some data and plot the *closed-form Linear Regressor*.

In [ ]:
# number of dataset points
N = 73 #@param {type:"slider", min:10, max:100, step:1}

X, y, coef = make_regression(n_samples = N, n_features=1, noise=20, coef=True)
closed_form = LinearRegression().fit(X, y)
cf_prediction = closed_form.predict(X)
plt.scatter(X, y)
plt.plot(X, cf_prediction, color='red', label='Closed form')
plt.legend(loc='lower right')

X = torch.Tensor(X)
y = torch.Tensor(y)

Now let's plot the Linear Regressor obtained by Gradient Descent, at each optimization step.

Feel free to play with the learning rate and total optimization steps and see how the results compare.

What happens if you set a (very) big learning rate? (>1)

In [ ]:
lr = 0.781 #@param {type: "slider", min: 0.001, max: 2, step: 0.005}
total_steps = 100 #@param {type:"slider", min: 0, max: 100, step: 1}

model = GDLinearRegression()
optimizer = GD(model.parameters(), lr=lr)
criterion = MSE()

fig, ax = plt.subplots()
plt.close()

gd_axis, = ax.plot([], [], label='Gradient Descent')
cf_axis, = ax.plot([], [], label='Closed Form')
legend = ax.legend(loc=2, prop={'size': 10})

def init():
  scatter = ax.scatter(X, y)
  cf_axis.set_data(X, cf_prediction)
  return (scatter, cf_axis,)

def train_and_plot(step: int):
  with torch.no_grad():
    y_pred = model(X)
  gd_axis.set_data(X, y_pred)
  
  train(model, X, y, optimizer, criterion)
  
  legend.texts[0].set_text(f"Gradient Descent (step={step})")
  legend.texts[1].set_text("Closed Form")

  return (gd_axis, legend, )

anim = animation.FuncAnimation(
    fig, train_and_plot, init_func=init, 
    frames=total_steps, interval=total_steps*2, blit=True
    )

rc('animation', html='jshtml')
anim

Let's see how the weight `w` and bias `b` changed after every Gradient Descent step.

In [ ]:
plt.plot(optimizer.w_hist, label='Weight', color='fuchsia')
plt.plot(optimizer.b_hist, label='Bias', color='green')
plt.xlabel('Gradient Descent steps')
plt.ylabel('Value')
plt.legend()

# Part II: Neural Networks, finally

## Defining some clusters

We want to train a model that can learn to classify 2D points sampled from 2 clusters in a plane, for example:

![Plot showing the distribution](https://i.imgur.com/XwavfqU.png)

These clusters are 2D Normal Distributions, $\mathcal{N}(\boldsymbol{\mu}, \boldsymbol{\Sigma})$, where $\boldsymbol{\mu} \in \mathbb{R}^2$ defines the center of the distribution, and $\boldsymbol{\Sigma} \in \mathbb{R}^{2 \times 2}$ is a diagonal matrix containing the standard deviations over each direction. In other words, $\boldsymbol{\Sigma} = \text{diag}(\boldsymbol{\sigma^2})$, where $\sigma_i^2 \in [0, +\infty)$ for all $i = \overline{1, 2}$ (See Chapter 2.3 - Bishop) [[2]](https://en.wikipedia.org/wiki/Multivariate_normal_distribution). 

Let's first define a helper function that will help us plot our 2D data, along with a class that implements a `sample` and a `plot` method.

In [ ]:
def plot_set(data: np.array, labels: np.array, alpha=1):
    """Helper function that plots labeled data"""
    plt.scatter(data[:,0], data[:,1], c=labels, cmap='Accent', alpha=alpha)

class LabeledDistribution:
    """This class represents a distribution over (example, label) pairs"""

    def sample(self, n: int) -> tuple:
        """Sample from the distribution"""
        raise NotImplementedError
        
    def plot(self, n: int = 1000):
        """Plot `n` values sampled from the distribution"""
        data, labels = self.sample(n)
        plot_set(data, labels.type(torch.float32))

Now let's define a class that helps us define and sample from clusters in a plane. In its initialiser, `mu`, `sigma` and `labels` are lists of the same length as the number of clusters. `mu[i]` and `sigma[i]` define a cluster of label `labels[i]`. In other words, they define a distribution $\mathcal{N}(\boldsymbol{\mu_i}, diag(\boldsymbol{\sigma_i}))$.

In [ ]:
class Clusters(LabeledDistribution):
    """This class defines labeled normal-distributed clusters in a plane with diagonal covariance matrix"""
    def __init__(self, mu: list = [[-2, -2], [2, 2]], 
                       sigma: list = [[1, 1], [1, 1]],
                       labels: list = [0, 1]):
        self._mu = torch.Tensor(mu)
        self._sigma = torch.Tensor(sigma)
        self._labels = torch.Tensor(labels).type(torch.long)

        self.no_cls = len(set(labels)) # the number of classes in our distribution
        self.no_clusters = self._mu.shape[0] # the number of clusters in our distribution
        
    def sample(self, n: int) -> tuple:
        data = torch.normal(torch.zeros(n, 2), torch.ones(n, 2)) # take n samples from a standard normal distribution
        cluster_idx = torch.randint(0, self.no_clusters, [n]) # randomly assign each observation to a cluster

        shifted_data = data * self._sigma[cluster_idx] + self._mu[cluster_idx] # transform each observation such that they are 
                                                                               # sampled from the distribution of the cluster
        labels = self._labels[cluster_idx] # get the label of each point by looking at the label of its cluster

        return shifted_data, labels

Now we can sample points from our clusters:

In [ ]:
dist = Clusters()
points, labels = dist.sample(10)
for point, label in zip(points, labels):
    x = round(point[0].item(), 2)
    y = round(point[1].item(), 2)
    print(f'({x}, {y}) belongs to class {label}')

Let's now plot our clusters to make sure we achieved what we wanted in the first place.

In [ ]:
# We now have a distribution to sample from :)
dist = Clusters()
dist.plot()

## Single Layer Neural Network

### Model definition
Let's build our first Neural Network in Pytorch, containing (for now) a single layer of perceptrons. Note that in the first example we manually construct our **Linear** layers (weights + biases) for didactic reasons, but you will use Pytorch's `torch.nn.Linear` module from now on.

<!-- First, recall that you can define a **perceptron** as:
$$y(x) = \sigma(\boldsymbol{w}^\intercal \boldsymbol{x} + b)$$ -->

First, recall that **a layer of perceptrons** can be expressed as:
$$\boldsymbol{f}(x) = \sigma(\boldsymbol{W} \boldsymbol{x} + \boldsymbol{b})$$

To build a Neural Network in Pytorch we subclass ``nn.Module`` and implement the following methods:
1. `__init__` (the initialiser), where we initialise our weights and biases, as well as define our activation function.
2. `forward`, which gets called under the hood when we run `model(...)`


### Using `nn.Module` to define Neural Networks
In practice, defining our Neural Networks by hand would be obviously cumbersome, as well as a huge waste of time. Therefore, we use `torch.nn` [[3]](https://pytorch.org/docs/stable/nn.html#torch.nn.Module) objects as helpers.

We can therefore define our Neural Network by deriving the `nn.Module` class, which will give us access to a range of methods, like `.forward()`, `.train()`, `.zero_grad()`, `.eval()`, `.parameters()` etc. For the individual layers, we will simply instantiate a `nn.Linear` [[4]](https://pytorch.org/docs/stable/nn.html?highlight=nn%20linear#torch.nn.Linear) object.

Even though we used our own class called `GD` before, in practice we use an optimiser instantiated from `torch.optim`, providing the model parameters, the learning rate and the regularization.



In [ ]:
class OneLayer(nn.Module):
  """
  Single-Layer Neural Network, without Pytorch nn.Linear Module
  """
  def __init__(self, 
               input_size: int, 
               output_size: int, 
               activation_fn = lambda x: torch.softmax(x,dim=-1)):
      super().__init__()

      # randomly initialise the weights with samples from a Gaussian distribution
      self._w = nn.Parameter(torch.randn([input_size, output_size], requires_grad = True))

      # ... and the biases with zeros
      self._b = nn.Parameter(torch.zeros([1, output_size], requires_grad = True))

      self._params = [self._w, self._b]
      
      # define the activation function
      self._activation_fn = activation_fn
      
  def forward(self, x: torch.Tensor) -> torch.Tensor:
      # σ(wx + b)
      return self._activation_fn(x @ self._w + self._b)

  def zero_grad(self):
      """Reset the gradients of the parameters"""
      for param in self._params:
          if param.grad is not None:
              param.grad.zero_()

In [ ]:
class OneLayer(nn.Module):
  """Single-Layer Neural Network using Pytorch nn.Linear"""
  def __init__(self, 
               input_size: int, 
               output_size: int, 
               activation_fn = lambda x: torch.softmax(x,dim=-1)):
      super().__init__()

      # we instantiate a Linear layer
      # look up nn.Linear in Pytorch documentation:
      #   https://pytorch.org/docs/stable/generated/torch.nn.Linear.html
      self.linear_tr = nn.Linear(
          in_features=input_size,
          out_features=output_size, 
          bias=True
      )
      
      # store the activation function
      self._activation_fn = activation_fn
      
  def forward(self, x: torch.Tensor) -> torch.Tensor:
      # step 1: apply linear layer to x
      output = self.linear_tr(x)

      # step 2: apply softmax to output
      y = self._activation_fn(output)

      return y

With the hard part out of the way, we now just define two helper functions:
1. `plot_decision` plots the decision boundary given a model, by predicting the class of every point in the plane
2. `plot_loss` simply plots the evolution of our loss function

In [ ]:
def plot_decision(model: nn.Module, bounds: list = [-8, 8, -8, 8],
                  detail: int = 100):
    """Plot the decision boundary of the `model`"""

    # Get all the points in the region we want to plot
    x1, x2 = torch.meshgrid(torch.linspace(bounds[0], bounds[1], detail),
                            torch.linspace(bounds[2], bounds[3], detail))
    
    data = torch.cat((x1.contiguous().view(-1, 1), x2.contiguous().view(-1, 1)),
                     dim=1)
    
    # use the model to predict the class of every point
    labels = torch.argmax(model(data), dim=-1)

    # plot all of the points
    plot_set(data, labels, alpha=0.1)
    
def plot_loss(loss: list, label: str, color: str = 'blue'):
    """Plot the evolution of the loss function"""
    plt.plot(loss, label=label, color=color)
    plt.legend()

Let's now plot the computation graph of our network, using a `in_size` of 2 (the `x` and `y` coordinates of every point), and an `out_size` of 2 (the scores the model attributes to the point belonging to each of the 2 classes). To predict the class of the point, we will simply take the `argmax` over the output of the network.

We can see from the computation graph all the operations being performed using the weights and biases.

In [ ]:
model = OneLayer(2,2)

x = torch.empty(2)
torchviz.make_dot(model(x))

Let's now plot the decision boundary of our randomly initialized model. Unless you are very lucky, the model won't explain the data very well:

In [ ]:
plot_decision(model)
dist.plot()

### Model training
It's finally time to train our model! First let's generate a sample from our distribution to use as training data.

In [ ]:
train_data, train_labels = dist.sample(100)
plot_set(train_data, train_labels)

Now let's define the loss function that we will use to train our model. We will manually define the Negative Log-Likelihood - NLL (though, in practice, this is already implemented in PyTorch).

The network outputs a tensor $\boldsymbol{p} = (p_1, p_2)$, representing the class likelihoods of each of the two classes. Let $p_i \in [0, 1]$ be the class score of the correct class. Then the NLL is defined as:
$$ \text{NLL} (\boldsymbol{p}) = - \log p_i $$

When calculating the loss over the whole data, we will simply compute the mean of the losses over every point.

In [ ]:
def NLL(output: torch.Tensor, true_labels: torch.Tensor) -> torch.Tensor:
    """
    Given the predictions of the neural network and the ground truth data,
    compute the negative log-likelihood.
    Arguments:
      output: output of neural network, tensor of size N x 2
      true_labels: ground truth data, Tensor of size N
    """

    # get the likelihoods of the correct class
    likelihood = output.gather(1,true_labels.view(-1,1))

    # calculate the mean of loss
    loss = -torch.log(likelihood)
    loss = torch.mean(loss)

    return loss

In [ ]:
NUM_EPOCHS = 400 # the number of epochs to train our model for
PRINT_EVERY = 100 # output the results every 100th epoch

train_loss = [] # we will store the loss function values at each epoch in this list

optim = GD(model.parameters(), lr=0.1) # we will use the `GD` class you defined in Exercise 1 as our optimiser

for i in range(NUM_EPOCHS):
    output = model(train_data) # run the model over the data to generate the class likelihoods
    
    loss = NLL(output, train_labels) # compute the loss over the current epoch
    
    loss.backward() # perform backprop over the loss to compute the gradients of the parameters
    with torch.no_grad():
      optim.step()      # run an optimisation step
      model.zero_grad() # reset the gradients, otherwise they will accumulate
    
    train_loss.append(loss.detach().numpy()) # store the loss of the current epoch

    if i % PRINT_EVERY == 0 or i == NUM_EPOCHS - 1: # every once in a while plot the decision boundary and print the loss
        print(f'EPOCH {i}:')
        print(f'loss = {loss.item()}')
        plot_decision(model)
        plot_set(train_data, train_labels)
        plt.show()
        
plot_loss(train_loss, 'train-loss')

As you can see, by using only one layer, the model is unable to learn decision boundaries more complex than a simple line. Therefore given the XOR distribution below, no `OneLayer` model can successfuly learn it.

Note that the XOR distribution is simply formed out of four clusters being assigned 2 different alternating classes.

In [ ]:
xor = Clusters(mu = [[-2, -2], [2, 2], [-2, 2], [2, -2]],
              sigma = [[1, 1], [1, 1], [1, 1], [1, 1]],
              labels = [0, 0, 1, 1])
xor.plot()

Because the two classes aren't linearly separable, a Neural Network with a single layer won't manage to learn the XOR distribution. But, for fun, let's actually check that this is true, by instantiating one and training it:

In [ ]:
model = OneLayer(2,2)
train_data, train_labels = xor.sample(100) # Let's sample our train data from the XOR distribution

NUM_EPOCHS = 400
PRINT_EVERY = 100

train_loss = []

optim = GD(model.parameters(), lr=0.1)

for i in range(NUM_EPOCHS):
    output = model(train_data)
    
    loss = NLL(output, train_labels)
    
    loss.backward()
    with torch.no_grad():
        optim.step()
        model.zero_grad()
    
    train_loss.append(loss.detach().numpy())

    if i % PRINT_EVERY == 0 or i == NUM_EPOCHS - 1:
        print(f'EPOCH {i}:')
        print(f'loss = {loss.item()}')
        plot_decision(model)
        plot_set(train_data, train_labels)
        plt.show()
        
plot_loss(train_loss, 'train-loss')

## Exercise 2: Learning the XOR distribution
Implement a multi-layer feed-forward network with 1 hidden layer, using the [``nn.Linear``](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html?highlight=nn%20linear#torch.nn.Linear) modules from Pytorch. This network is capable of learning the XOR distribution. Inspect the decision boundary and the evolution of the loss over the epochs.

In [ ]:
class MultiLayer(nn.Module):
  """Multi-Layer Neural Network (1 hidden layer), without using nn.Linear"""
  def __init__(self, 
               input_size: int, 
               hidden_size: int, 
               output_size: int, 
               hidden_activation_fn = nn.Tanh(),
               output_activation_fn = nn.Softmax(dim=-1)):
      """
      TODO: initialize weights and biases for the hidden and the output layers
      Arguments:
        input_size: number of input neurons
        hidden_size: number of neurons of hidden layer 
        output_size: number of neurons of output layer
      """
      super().__init__()

      # Now we'll have two sets of weights and biases, corresponding to two layers
      # 1st layer
      self._w1 = nn.Parameter(torch.randn([input_size, hidden_size], requires_grad = True))
      self._b1 = nn.Parameter(torch.zeros([1, hidden_size], requires_grad = True))
      
      # 2nd layer
      self._w2 = nn.Parameter(torch.randn([hidden_size, output_size], requires_grad = True))
      self._b2 = nn.Parameter(torch.zeros([1, output_size], requires_grad = True))

      self._params = [self._w1, self._w2, self._b1, self._b2]

      self._hidden_activation_fn = hidden_activation_fn
      self._output_activation_fn = output_activation_fn
      
  def forward(self, x: torch.Tensor) -> torch.Tensor:
      """
      Feedforward through the two layers.
      Arguments:
        x: tensor of size (number_of_examples x self.input_size)
      
      Returns a tensor of size self.output_size, which is the output of the
      network after the softmax activation.
      """
      # Layer 1
      x = x @ self._w1 + self._b1
      x = self._hidden_activation_fn(x)

      # Layer 2
      x = x @ self._w2 + self._b2
      x = self._output_activation_fn(x)

      return x
  
  def zero_grad(self):
      """Reset the gradients of the parameters"""
      for param in self._params:
          if param.grad is not None:
              param.grad.zero_()

In [ ]:
class MultiLayer(nn.Module):
  """TODO: Multi-Layer Neural Network (1 hidden layer), using nn.Linear"""
  def __init__(self, 
               input_size: int, 
               hidden_size: int, 
               output_size: int, 
               hidden_activation_fn = nn.Tanh(),
               output_activation_fn = nn.Softmax(dim=-1)):
      """
      TODO: initialize weights and biases for the hidden and the output layers
      Arguments:
        input_size: number of input neurons
        hidden_size: number of neurons of hidden layer 
        output_size: number of neurons of output layer
      """
      super().__init__()

      # TODO: initialize the first linear layer using nn.Linear
      self._fst_linear = ...
      
      # TODO: initialize the second linear layer using nn.Linear
      self._snd_linear = ...

      self._hidden_activation_fn = hidden_activation_fn
      self._output_activation_fn = output_activation_fn
      
  def forward(self, x: torch.Tensor) -> torch.Tensor:
      """
      Feedforward through the two layers.
      Arguments:
        x: tensor of size (number_of_examples x self.input_size)
      
      Returns a tensor of size self.output_size, which is the output of the
      network after the softmax activation.
      """
      # TODO: apply first linear layer, then activation
      h = ...

      # TODO: apply second linear layer, then activation
      o = ...
      
      return o

In [ ]:
def NLL(output: torch.Tensor, true_labels: torch.Tensor) -> torch.Tensor:
    """
    Given the predictions of the neural network and the ground truth data,
    compute the negative log-likelihood.
    Arguments:
      output: output of neural network, tensor of size N x 2
      true_labels: ground truth data, Tensor of size N
    """
    likelihood = output.gather(1, true_labels.view(-1,1))

    loss = -torch.log(likelihood)
    loss = torch.mean(loss)

    return loss

In [ ]:
# We need to perform Gradient Descent on 2 layers now, so let's redefine our `GD` class
class GD:
  def __init__(self, params: torch.Tensor, lr: int):
    self.w1, self.w2, self.b1, self.b2 = list(params)
    self.lr = lr

  def step(self):
    with torch.no_grad():
      self.w1 -= self.lr * self.w1.grad
      self.b1 -= self.lr * self.b1.grad
      self.w2 -= self.lr * self.w2.grad
      self.b2 -= self.lr * self.b2.grad

In [ ]:
# For fun, use `torchviz.make_dot()` to see the computation graph of the network
INPUT_SIZE = 2
HIDDEN_SIZE = 4
OUTPUT_SIZE = 2
x = torch.randn(INPUT_SIZE)
model = MultiLayer(input_size=INPUT_SIZE, 
                   hidden_size=HIDDEN_SIZE,
                   output_size=OUTPUT_SIZE)
torchviz.make_dot(model(x))

In [ ]:
# Finally, train the network and plot the decision boundaries
model = MultiLayer(input_size=INPUT_SIZE, 
                   hidden_size=HIDDEN_SIZE,
                   output_size=OUTPUT_SIZE)
train_data, train_labels = xor.sample(100) # Let's sample our train data from the XOR distribution

NUM_EPOCHS = 400
PRINT_EVERY = 100

train_loss = []
optim = GD(model.parameters(), lr=0.1)

for i in range(NUM_EPOCHS):
    output = model(train_data)
    loss = NLL(output, train_labels)
    
    loss.backward()
    optim.step()
    model.zero_grad()
    
    train_loss.append(loss.detach().numpy())

    if i % PRINT_EVERY == 0 or i == NUM_EPOCHS - 1:
        print(f'EPOCH {i}:')
        print(f'loss = {loss.item()}')
        plot_decision(model)
        plot_set(train_data, train_labels)
        plt.show()
        
plot_loss(train_loss, 'train-loss')

## Exercise 3: Learning three clusters

Define a Neural Network that can successfully classify samples belonging to these three clusters:

![3 clusters in a plane](https://i.imgur.com/EnFKfZG.png)

First up, let's define our clusters:


In [ ]:
threeClass = Clusters(mu = [[-2,-2],[2,2],[-2,2]],
                      sigma = [[1,1],[1,1],[1,1]],
                      labels = [0,1,2])
threeClass.plot()

Finally, let's sample a train dataset and a validation dataset:

In [ ]:
train_data, train_labels = threeClass.sample(200)
val_data, val_labels = threeClass.sample(50)

plot_set(train_data, train_labels)

You should:
1. Define the Neural Network, inheriting `torch.nn.Module` and using all the appropriate objects provided by torch. You can use one of the loss functions already provided by `torch.nn.functional` [[5]](https://pytorch.org/docs/stable/nn.functional.html);
2. For fun, plot the computation graph;
3. Train the network like before, printing the loss and decision boundary from time to time. For now, you can use the `torch.optim.SGD` optimiser [[6]](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD);
4. Take a sample from the distribution and use it as a validation dataset, testing the loss function on it without training every epoch. Plot both losses;
5. Compare the model's training using `torch.optim.SGD` vs. `torch.optim.Adam` [[7]](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam).

In [ ]:
class ThreeClassNN(nn.Module):
    def __init__(self, 
                 input_size: int, 
                 hidden_size: int, 
                 output_size: int,
                 hidden_activation_fn = nn.ReLU()):
        """
        TODO: initialize a multi-layer NN with 1 hidden layer using the
        pytorch package torch.nn.Linear
        """
        # Initialise the base class (nn.Module)
        super().__init__()

        # TODO use `torch.nn.Linear` to instantiate the hidden and the output
        # layer. Look up the Pytorch documentation:
        #   https://pytorch.org/docs/stable/generated/torch.nn.Linear.html

        self._layer1 = ...
        self._layer2 = ...

        self._hidden_activation = hidden_activation_fn
        
    def forward(self, x):
        # TODO: apply first layer transformation + activation
        h = ...

        # TODO: apply second layer transformation
        # Because we will use CrossEntropy as our loss, we don't need a
        # a softmax activation function after layer 2
        out = ...
        
        return out

In [ ]:
# For fun, use `torchviz.make_dot()` to see the computation graph of the network
model = ThreeClassNN(2, 4, 3)
x = torch.randn(2)
torchviz.make_dot(model(x))

In [ ]:
# Finally, train the network and plot the decision boundaries
model = ThreeClassNN(2, 4, 3)

NUM_EPOCHS = 400
PRINT_EVERY = 50

train_loss = []
val_loss = [] # This time we will track the loss on val_data

# TODO: instantiante SGD optimizer with a learning rate of 0.01
# look up the Pytorch documentation:
# https://pytorch.org/docs/stable/optim.html?highlight=sgd#torch.optim.SGD
optim = ...

# TODO: instantiate the Cross Entropy loss
# look up the Pytorch documentation:
# https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
criterion = ...

for i in range(NUM_EPOCHS):
    # Set the model to train mode and reset the gradients
    model.train()
    optim.zero_grad()

    output = model(train_data)
    
    # The main difference between Cross Entropy and NLL is that the first doesn't
    # expect the output to be class likelihoods (\in [0, 1]), but rather
    # class scores (\in \mathbb{R}). That's why we didn't use softmax this time
    # on the last layer.
    loss = criterion(output, train_labels)
    
    loss.backward()
    optim.step()
    
    train_loss.append(loss.detach().numpy())

    if i % PRINT_EVERY == 0 or i == NUM_EPOCHS - 1:
        print(f'EPOCH {i}:')
        print(f'loss = {loss.item()}')
        plot_decision(model)
        plot_set(train_data, train_labels)
        plt.show()

    # Every epoch, let's evaluate our model on the validation data and plot both losses at the end
    model.eval() # set the model to evaluation mode
    with torch.no_grad():
        output = model(val_data)
        validation_loss = F.cross_entropy(output, val_labels)
        val_loss.append(validation_loss)

plot_loss(train_loss, 'train-loss')
plot_loss(val_loss, 'val-loss', color='green')